In [1]:
%load_ext autoreload
%autoreload 2

# Goal: Run facility choice causal model optimization to find input parameters for Vivarium

The optimization results are output in a `.csv` file at the end of the notebook.

## Requirements:

If you want to run this notebook or any of the code in the
`birth_facility.py` or `solution_finding.py` modules, you will need the right environment.
You can create it with these steps (on 64-bit Linux):

- `conda create --name facility_choice --file environment_lock_conda.txt`
- `conda activate facility_choice`
- `pip install -r environment_lock_pip.txt`

If you're not on 64-bit Linux, replacing the first command with `conda create --name facility_choice python=3.10`
should approximately recreate the environment.

If you want to update all the packages in the environment, create it using `conda env create --name facility_choice --file environment.yaml` (no need for any `pip` commands).
Then, run the following commands inside the environment you created to update the lock files:

- `conda list --explicit > environment_lock_conda.txt`
- `pip freeze | grep -v 'file:///' | grep -v '\-e' > environment_lock_pip.txt`

You will also need to be on the IHME cluster to run this notebook, as the causal model
nanosim loads LBWSG data from an Artifact for the MNCNH Portfolio model.

In [2]:
import pandas as pd

import birth_facility as bf
import solution_finding as sf

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e scipy -e statsmodels -e matplotlib -e gbd

Fri Dec 12 11:34:31 PST 2025
lutzes
Linux long-slurm-sarchive-p0033 5.4.0-216-generic #236-Ubuntu SMP Fri Apr 11 19:53:21 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/lutzes/vivarium_gates_mncnh/src/vivarium_gates_mncnh/data/facility_choice
Python 3.10.19
facility_choice       *  /ihme/homes/lutzes/.conda/envs/facility_choice
gbd                       4.37.1                   pypi_0    pypi
gbd-mapping               5.0.0                    pypi_0    pypi
matplotlib-inline         0.1.7                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    1.5.3                    pypi_0    pypi
pandas-stubs              2.2.3.250308             pypi_0    pypi
scipy                     1.15.3                   pypi_0    pypi
statsmodels               0.14.5                   pypi_0    pypi


# Load data

The input data comes from two sources:

- An artifact for the `vivarium_gates_mncnh` simulation, which is used for population structure, LBWSG exposure and RRs, ANC and IFD coverage, etc.
- The file `facility_choice_data.xlsx` in this folder (downloaded from
  the [master version on
  Sharepoint](https://uwnetid.sharepoint.com/:x:/r/sites/ihme_simulation_science_team/Shared%20Documents/Research/BMGF_MNCH/MNCNH%20portfolio%20products/01_Planning/facility_choice_data.xlsx?d=wf3b3dd5f641f413ba3537c4ca3364cdf&csf=1&web=1&e=cNf9X7)),
  which contains all the other country-specific data to run the model

In [3]:
data_e = bf.BirthFacilityChoiceData('ethiopia')
data_n = bf.BirthFacilityChoiceData('nigeria')
data_p = bf.BirthFacilityChoiceData('pakistan')

# Create models for all 3 countries

Each of the three model objects defines the causal model nanosim that
will be run repeatedly by the `OptimalSolutionFinder.find_solutions`
optimization routine below.

In [4]:
# Choose a population size that's large enough to get stochastically
# stable results, but small enough that the optimization isn't too slow
pop_size = 100_000
# In Vivarium, we will be using the 2-facility-type model (at-home vs.
# in-facility)
num_facility_types = 2

# The random seeds were generated by calling numpy.random.SeedSequence()
# and then copying the generated entropy
model_e = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_e, pop_size, num_facility_types,
    seed=97269740763240770980627172105157383126
)
model_n = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_n, pop_size, num_facility_types,
    seed=230426788122306909803111949239871602900
)
model_p = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_p, pop_size, num_facility_types,
    seed=150031875166580400207947577126615739298
)

# Create solution finder objects and specify inputs for optimization

In [5]:
# These are the 5 parameters we need to solve for in the optimization
model_e.parameter_data

attribute,correlated_pair,bounds,position
parameter_name,,,
"corr(anc, lbwsg_category)","(anc, lbwsg_category)","(-1, 1)",0
"corr(anc, facility)","(anc, facility)","(-1, 1)",1
"corr(lbwsg_category, facility)","(lbwsg_category, facility)","(-1, 1)",2
prob_home_given_believed_preterm,<NA>,"(0, 1)",3
prob_home_given_believed_term,<NA>,"(0, 1)",4


In [6]:
finder_e = sf.OptimalSolutionFinder(model_e)
finder_n = sf.OptimalSolutionFinder(model_n)
finder_p = sf.OptimalSolutionFinder(model_p)

# 25 initial conditions seems to be sufficient to get close to the
# global minimum
n_points = 25
# Fix the two LBWSG correlations at 0.2 -- According to the above
# parameter_data dataframe, these parameters are in positions 0 and 2 in
# the parameter vector
fixed_x_components = {0: 0.2, 2: 0.2}

# Find solutions

# Ethiopia

In [7]:
%%time
solutions_e = finder_e.find_solutions(n_points, fixed_x_components)
print(solutions_e.sorted_losses)
solutions_e.sorted_x_values

[1.7723026546345721e-07, 1.9952510399434686e-07, 2.0477719742117984e-07, 2.218135581877334e-07, 3.078801503741957e-07, 3.450787386949372e-07, 3.7275457842600446e-07, 3.9806862572877577e-07, 5.700400668162686e-07, 6.900133951992515e-07, 7.856345348722371e-07, 1.3295876702956733e-06, 2.923371415941567e-06, 1.2997157871841303e-05, 0.0027314352777229045, 0.007202935262415222, 0.03673730868003555, 0.050179029875546344, 0.05067239458674899, 0.053649192898360454, 0.05398304365097828, 0.055141997143512866, 0.17555767089205843, 0.1824441614623713, inf]
CPU times: user 41min 34s, sys: 41min 44s, total: 1h 23min 18s
Wall time: 11min 21s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.689604,0.2,0.297958,0.504472
1,0.2,0.689647,0.2,0.297390,0.505320
2,0.2,0.689766,0.2,0.297482,0.505202
3,0.2,0.689660,0.2,0.297050,0.504917
4,0.2,0.691868,0.2,0.300518,0.506123
5,0.2,0.688936,0.2,0.298277,0.504484
6,0.2,0.690589,0.2,0.297442,0.504552
7,0.2,0.688747,0.2,0.299426,0.504156
8,0.2,0.690068,0.2,0.297357,0.503855
9,0.2,0.690326,0.2,0.298981,0.503952


In [8]:
data_e.targets_2_facility_types.to_frame().join(solutions_e.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.183369  0.183596  0.186295   
               anc1                     0.664786  0.665065  0.665198   
preterm        at_home                  0.163579  0.163640  0.163469   
               in_facility              0.163486  0.162907  0.163971   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.183165  0.186249  0.182726  0.183792  0.183057   
               anc1         0.664630  0.665377  0.664028  0.665147  0.665749   
preterm        at_home      0.163394  0.163176  0.163734  0.163579  0.163319   
               in_facility  0.163101  0.164201  0.163816  0.162956  0.163178   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.183464  0.183704  ...  0.159861  0.221160   
               anc1         0.664784  0.665574  ...  0.697064  0.797368   
preterm        at_home      0.163876  0.163346  ...  0.101920  0.321191   
               in_facility  0.162721  0.163157  ...  0.208519  0.086413   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.375137  0.186662  0.167538  0.194404  0.191927   
               anc1         0.707197  0.520748  0.598031  0.607024  0.552448   
preterm        at_home      0.046791  0.267625  0.298572  0.305518  0.283943   
               in_facility  0.231031  0.035776  0.029541  0.029632  0.029769   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.620721  0.371736  0.183596  
               anc1         0.697941  0.279670  0.665065  
preterm        at_home      0.434987  0.035292  0.163640  
               in_facility  0.035967  0.457911  0.162907  

[4 rows x 26 columns]

In [9]:
data_e.input_probabilities.to_frame().join(solutions_e.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.515312,0.516510,0.515200,0.516510,0.515200,0.515200,0.516510,0.51651,0.516510,0.516510,...,0.515200,0.512060,0.515200,0.512060,0.516510,0.516510,0.51206,0.515200,0.513610,0.516510
Female,0.484688,0.483490,0.484800,0.483490,0.484800,0.484800,0.483490,0.48349,0.483490,0.483490,...,0.484800,0.487940,0.484800,0.487940,0.483490,0.483490,0.48794,0.484800,0.486390,0.483490
anc0,0.219890,0.219340,0.219330,0.219310,0.219330,0.219400,0.219270,0.21933,0.219280,0.219320,...,0.219440,0.219570,0.219440,0.219220,0.219890,0.219800,0.21951,0.220550,0.219080,0.219340
anc1,0.780110,0.780660,0.780670,0.780690,0.780670,0.780600,0.780730,0.78067,0.780720,0.780680,...,0.780560,0.780430,0.780560,0.780780,0.780110,0.780200,0.78049,0.779450,0.780920,0.780660
preterm,0.163527,0.163230,0.163750,0.163230,0.163750,0.163780,0.163230,0.16324,0.163230,0.163240,...,0.163660,0.163690,0.163660,0.163870,0.163150,0.163070,0.16364,0.163290,0.162010,0.163230
term,0.836473,0.836770,0.836250,0.836770,0.836250,0.836220,0.836770,0.83676,0.836770,0.836760,...,0.836340,0.836310,0.836340,0.836130,0.836850,0.836930,0.83636,0.836710,0.837990,0.836770
at_home,0.441073,0.440540,0.439840,0.440960,0.439710,0.441570,0.440400,0.44012,0.440760,0.440110,...,0.420820,0.329150,0.365670,0.552490,0.496630,0.483670,0.52669,0.319090,0.700160,0.440540
in_facility,0.558927,0.559460,0.560160,0.559040,0.560290,0.558430,0.559600,0.55988,0.559240,0.559890,...,0.579180,0.670850,0.634330,0.447510,0.503370,0.516330,0.47331,0.680910,0.299840,0.559460
BEmONC,0.089922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Nigeria

In [10]:
%%time
solutions_n = finder_n.find_solutions(n_points, fixed_x_components)
print(solutions_n.sorted_losses)
solutions_n.sorted_x_values

[1.803584763759858e-07, 1.9528262307222377e-07, 1.9796745431843732e-07, 2.402485271835175e-07, 2.910207956841049e-07, 3.268527057898041e-07, 3.3379011421530436e-07, 3.8834574334067895e-07, 3.935646626374023e-07, 4.091682669349339e-07, 4.1598149513877303e-07, 4.848862317219371e-07, 5.178593408539101e-07, 5.470255457495909e-07, 0.0002707776493284797, 0.002768706453943448, 0.017379117696565305, 0.020479693685886202, 0.0213077575481736, 0.023077541903608556, 0.043251461122949086, 0.04347411550346725, 0.043481522417392116, 0.050802703415321004, 0.12420989040872188]
CPU times: user 39min 19s, sys: 38min 29s, total: 1h 17min 49s
Wall time: 11min 14s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.417728,0.2,0.374638,0.533572
1,0.2,0.418261,0.2,0.375410,0.532958
2,0.2,0.417307,0.2,0.375967,0.533468
3,0.2,0.419124,0.2,0.375863,0.533322
4,0.2,0.416397,0.2,0.375610,0.533189
5,0.2,0.418296,0.2,0.373833,0.534031
6,0.2,0.420855,0.2,0.374848,0.532775
7,0.2,0.420794,0.2,0.375612,0.532534
8,0.2,0.421863,0.2,0.375094,0.532392
9,0.2,0.421526,0.2,0.375106,0.532221


In [11]:
data_n.targets_2_facility_types.to_frame().join(solutions_n.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.309069  0.308949  0.309988   
               anc1                     0.584168  0.584159  0.584678   
preterm        at_home                  0.172663  0.172719  0.172600   
               in_facility              0.153000  0.153419  0.153536   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.310066  0.309530  0.310652  0.308860  0.307689   
               anc1         0.583965  0.582964  0.584123  0.584099  0.585274   
preterm        at_home      0.172563  0.173220  0.172594  0.172551  0.172983   
               in_facility  0.153514  0.153267  0.153572  0.153557  0.153150   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.307249  0.308327  ...  0.310745  0.435978   
               anc1         0.585187  0.585855  ...  0.620737  0.452426   
preterm        at_home      0.173124  0.172596  ...  0.137488  0.180902   
               in_facility  0.152990  0.153383  ...  0.184105  0.141407   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.244683  0.452921  0.457985  0.293385  0.311443   
               anc1         0.644143  0.444799  0.436385  0.525586  0.532169   
preterm        at_home      0.080691  0.189127  0.187435  0.279262  0.283889   
               in_facility  0.232590  0.130482  0.133299  0.029467  0.029905   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.318217  0.320998  0.628819  
               anc1         0.538785  0.646929  0.687096  
preterm        at_home      0.287083  0.332853  0.428110  
               in_facility  0.030052  0.031132  0.035097  

[4 rows x 26 columns]

In [12]:
data_n.input_probabilities.to_frame().join(solutions_n.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.512450,0.510960,0.510960,0.510960,0.509000,0.510960,0.510960,0.510960,0.510960,0.510960,...,0.512480,0.510960,0.51248,0.51248,0.511540,0.512480,0.509000,0.509000,0.510960,0.511540
Female,0.487550,0.489040,0.489040,0.489040,0.491000,0.489040,0.489040,0.489040,0.489040,0.489040,...,0.487520,0.489040,0.48752,0.48752,0.488460,0.487520,0.491000,0.491000,0.489040,0.488460
anc0,0.257001,0.257000,0.257010,0.257010,0.256970,0.257040,0.257010,0.256980,0.256990,0.257000,...,0.257060,0.256320,0.25768,0.25627,0.257170,0.256830,0.256740,0.256680,0.257260,0.256640
anc1,0.742999,0.743000,0.742990,0.742990,0.743030,0.742960,0.742990,0.743020,0.743010,0.743000,...,0.742940,0.743680,0.74232,0.74373,0.742830,0.743170,0.743260,0.743320,0.742740,0.743360
preterm,0.162567,0.162810,0.162800,0.162780,0.162990,0.162820,0.162800,0.162790,0.162780,0.162710,...,0.162710,0.163200,0.16290,0.16292,0.163510,0.162870,0.163120,0.163150,0.162960,0.163950
term,0.837433,0.837190,0.837200,0.837220,0.837010,0.837180,0.837200,0.837210,0.837220,0.837290,...,0.837290,0.836800,0.83710,0.83708,0.836490,0.837130,0.836880,0.836850,0.837040,0.836050
at_home,0.486533,0.486570,0.485920,0.486430,0.487300,0.486170,0.486640,0.486060,0.486240,0.485470,...,0.458950,0.551790,0.45879,0.55312,0.558060,0.534050,0.524500,0.517830,0.436920,0.327860
in_facility,0.513467,0.513430,0.514080,0.513570,0.512700,0.513830,0.513360,0.513940,0.513760,0.514530,...,0.541050,0.448210,0.54121,0.44688,0.441940,0.465950,0.475500,0.482170,0.563080,0.672140
BEmONC,0.002271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pakistan

In [13]:
%%time
solutions_p = finder_p.find_solutions(n_points, fixed_x_components)
print(solutions_p.sorted_losses)
solutions_p.sorted_x_values

[1.2130272376564832e-06, 1.4064090247956074e-06, 1.4632990149587854e-06, 1.6718175408492186e-06, 1.735291734639155e-06, 1.7496484748802388e-06, 1.7578073802093641e-06, 1.7809508513488836e-06, 1.791798600048189e-06, 1.90462481430842e-06, 1.94481400439539e-06, 2.007714933749405e-06, 2.0308393150481052e-06, 2.040183569529397e-06, 2.0516404927173326e-06, 2.2401339279110033e-06, 4.75351837880833e-06, 0.0007671429189125556, 0.0009371048823318961, 0.004282701903829178, 0.0058606736580609775, 0.010704393759763398, 0.056978332410484334, 0.06751776034056767, inf]
CPU times: user 42min 55s, sys: 43min 2s, total: 1h 25min 58s
Wall time: 11min 50s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.351232,0.2,0.165997,0.250153
1,0.2,0.350558,0.2,0.165860,0.250654
2,0.2,0.352378,0.2,0.165408,0.249522
3,0.2,0.351767,0.2,0.165962,0.250641
4,0.2,0.352643,0.2,0.165701,0.249127
5,0.2,0.350861,0.2,0.166809,0.250819
6,0.2,0.350525,0.2,0.166374,0.251116
7,0.2,0.352605,0.2,0.165409,0.249013
8,0.2,0.353620,0.2,0.165843,0.249013
9,0.2,0.353958,0.2,0.165087,0.249193


In [14]:
data_p.targets_2_facility_types.to_frame().join(solutions_p.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.556322  0.558980  0.558507   
               anc1                     0.795407  0.795365  0.794757   
preterm        at_home                  0.194529  0.194803  0.194144   
               in_facility              0.155164  0.153542  0.153712   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.555615  0.555497  0.558726  0.555659  0.557843   
               anc1         0.795882  0.795086  0.795988  0.794651  0.794303   
preterm        at_home      0.193772  0.193936  0.194916  0.196069  0.194228   
               in_facility  0.154247  0.154136  0.153552  0.153297  0.153651   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.559257  0.558461  ...  0.553771  0.556397   
               anc1         0.796280  0.796117  ...  0.795324  0.793323   
preterm        at_home      0.194963  0.195275  ...  0.193566  0.194557   
               in_facility  0.153555  0.153480  ...  0.154260  0.153941   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.585973  0.558069  0.610149  0.552454  0.560174   
               anc1         0.774171  0.814712  0.786025  0.872535  0.867099   
preterm        at_home      0.202984  0.167180  0.142486  0.186470  0.120517   
               in_facility  0.150455  0.161736  0.169328  0.158415  0.170328   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.518290  0.641622  0.556397  
               anc1         0.822242  0.736040  0.793323  
preterm        at_home      0.493132  0.473203  0.194557  
               in_facility  0.080269  0.050254  0.153941  

[4 rows x 26 columns]

In [15]:
data_p.input_probabilities.to_frame().join(solutions_p.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.519811,0.519420,0.519420,0.517320,0.517320,0.519420,0.515460,0.519420,0.519420,0.519420,...,0.517320,0.517320,0.515460,0.517320,0.515460,0.517320,0.517320,0.517320,0.515460,0.517320
Female,0.480189,0.480580,0.480580,0.482680,0.482680,0.480580,0.484540,0.480580,0.480580,0.480580,...,0.482680,0.482680,0.484540,0.482680,0.484540,0.482680,0.482680,0.482680,0.484540,0.482680
anc0,0.075919,0.075280,0.075290,0.075340,0.075320,0.075350,0.075280,0.075290,0.075350,0.075350,...,0.075320,0.075270,0.075140,0.075600,0.075080,0.075590,0.075780,0.076270,0.075730,0.075270
anc1,0.924081,0.924720,0.924710,0.924660,0.924680,0.924650,0.924720,0.924710,0.924650,0.924650,...,0.924680,0.924730,0.924860,0.924400,0.924920,0.924410,0.924220,0.923730,0.924270,0.924730
preterm,0.163933,0.162720,0.162730,0.163030,0.163010,0.162730,0.162850,0.162720,0.162730,0.162750,...,0.163020,0.163060,0.163060,0.162850,0.163230,0.162670,0.162550,0.163230,0.164920,0.163060
term,0.836067,0.837280,0.837270,0.836970,0.836990,0.837270,0.837150,0.837280,0.837270,0.837250,...,0.836980,0.836940,0.836940,0.837150,0.836770,0.837330,0.837450,0.836770,0.835080,0.836940
at_home,0.222744,0.222430,0.223030,0.222220,0.222960,0.221890,0.223340,0.223500,0.221580,0.221790,...,0.222870,0.224510,0.239970,0.204690,0.227180,0.151660,0.156160,0.200940,0.271110,0.224510
in_facility,0.777256,0.777570,0.776970,0.777780,0.777040,0.778110,0.776660,0.776500,0.778420,0.778210,...,0.777130,0.775490,0.760030,0.795310,0.772820,0.848340,0.843840,0.799060,0.728890,0.775490
BEmONC,0.264677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Collect optimization results from the 3 countries

Select the solution with the lowest log loss for each country. The above
solutions dataframes are already sorted by log loss, so we just select
the first row of each.

In [16]:
solutions_by_location = {
    'Ethiopia': solutions_e,
    'Nigeria': solutions_n,
    'Pakistan': solutions_p,
}
best_solutions = pd.concat({
    location: solutions.sorted_x_values.loc[0]
    for location, solutions in solutions_by_location.items()
}, axis=1)
best_solutions

,Ethiopia,Nigeria,Pakistan
parameter_name,,,
"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
"corr(anc, facility)",0.689604,0.417728,0.351232
"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
prob_home_given_believed_preterm,0.297958,0.374638,0.165997
prob_home_given_believed_term,0.504472,0.533572,0.250153


# Save results to file

In [18]:
output_filename = 'facility_choice_optimization_results.csv'
best_solutions.to_csv(output_filename)
pd.read_csv(output_filename)

,parameter_name,Ethiopia,Nigeria,Pakistan
0,"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
1,"corr(anc, facility)",0.689604,0.417728,0.351232
2,"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
3,prob_home_given_believed_preterm,0.297958,0.374638,0.165997
4,prob_home_given_believed_term,0.504472,0.533572,0.250153
